In [1]:
from typing import Dict, List
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


def load_ecg_dataset(batch_size: int):
    # Implement dataset loading logic here
    # You should return a generator that yields (inputs, labels) tuples
    pass


def get_model(embedding_dim: int, hidden_dim: int, num_classes: int, dropout: float) -> Model:
    """Get PCLR embedding model and finetune it"""
    # Load PCLR model
    base_model = load_model("./PCLR.h5")
    base_model.trainable = False

    # Create finetune classifier model
    inputs = Input(shape=(4096, 12))
    x = base_model(inputs, training=False)
    x = Dense(hidden_dim, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    # Combine base model and classifier model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

    return model


def finetune_model(model: Model, batch_size: int, epochs: int):
    # Load ECG dataset
    dataset = load_ecg_dataset(batch_size)

    # Finetune the model on the ECG dataset
    model.fit(dataset, epochs=epochs)

    return model


# Hyperparameters and finetuning
args = {
    "embedding_dim": 512,
    "hidden_dim": 256,
    "num_classes": 5,
    "dropout": 0.5,
    "batch_size": 32,
    "epochs": 10,
}

model = get_model(
    embedding_dim=args["embedding_dim"],
    hidden_dim=args["hidden_dim"],
    num_classes=args["num_classes"],
    dropout=args["dropout"],
)

finetuned_model = finetune_model(model, batch_size=args["batch_size"], epochs=args["epochs"])

ModuleNotFoundError: No module named 'tensorflow'